In [11]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import Sequence
from sklearn.cluster import KMeans
from tqdm import tqdm
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import math
import sklearn.metrics as sm
import cv2

In [12]:
# Function to Extract features from the images
def image_feature(direc):
    model = InceptionV3(weights='imagenet', include_top=False)
    features = []
    img_name = []
    for i in tqdm(direc):
        fname='./archive/train/train1/'+i
        img=image.load_img(fname,target_size=(224,224))
        x = img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        feat=model.predict(x)
        feat=feat.flatten()
        features.append(feat)
        img_name.append(i)
    return features,img_name

In [13]:
data = []
label = []
path = "./archive/train/train1/"
cat = 0
dog = 0
IMG_SIZE = 224

for file in os.listdir(path):
    img=cv2.imread(path+file)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img=img.astype('float32')
    
    if file[:3]=='cat':
        if cat==4: continue
        cat+=1
        label.append("cat")
    else:
        if dog==4: continue
        dog+=1
        label.append("dog")
    
    data.append(img)
data = np.array(data)

In [14]:
def get_reference_dict(clusters,data_label):
    reference_label = {}
    # run一遍簇的每個標籤
    for i in range(len(np.unique(clusters))):
        index = np.where(clusters == i,1,0)
        num = np.bincount(data_label[index==1]).argmax()
        reference_label[i] = num
    return reference_label

In [ ]:
data_label = []
for i in label:
    if i=="cat": data_label.append(0)
    else: data_label.append(1)
data_label = np.array(data_label)

In [ ]:
k = 2
clusters = KMeans(k, random_state = 1400)
clusters.fit(data)